In [2]:
import sys
import os
import cv2
import jetbot
import numpy as np
import traitlets
import control_utils
from ipywidgets import Image, VBox
from IPython.display import display
from jetbot import bgr8_to_jpeg, Camera

print(sys.executable)
print(cv2.__file__)
print(control_utils.__file__)
print(jetbot.__file__)

/home/jetbot/.pyenv/versions/3.11.6/bin/python
Wersja Pythona: 3.11.6 (main, Dec 31 2024, 02:53:33) [GCC 7.5.0]
/home/jetbot/.pyenv/versions/3.11.6/lib/python3.11/site-packages/cv2.cpython-311-aarch64-linux-gnu.so
/home/jetbot/skarpety/src/control_utils.py
/home/jetbot/.pyenv/versions/3.11.6/lib/python3.11/site-packages/jetbot


In [7]:
# pelen obraz z kamerki 


print("Inicjalizacja kamery")
camera = Camera.instance(width=300, height=300)

print("Widget Image do wyświetlania obrazu")
image_widget = Image(format='jpeg', width=300, height=300)

print("Wiązanie dynamicznego obrazu z widgetem")
traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)

print("Wyświetlenie widgetu w Jupyter Notebook")
display(VBox([image_widget]))

Inicjalizacja kamery
GST_ARGUS: Cleaning up
CONSUMER: Done Success
GST_ARGUS: Done Success
GST_ARGUS: Creating output stream
CONSUMER: Waiting until producer is connected...
GST_ARGUS: Available Sensor modes :
GST_ARGUS: 3264 x 2464 FR = 21.000000 fps Duration = 47619048 ; Analog Gain range min 1.000000, max 10.625000; Exposure Range min 13000, max 683709000;

GST_ARGUS: 3264 x 1848 FR = 28.000001 fps Duration = 35714284 ; Analog Gain range min 1.000000, max 10.625000; Exposure Range min 13000, max 683709000;

GST_ARGUS: 1920 x 1080 FR = 29.999999 fps Duration = 33333334 ; Analog Gain range min 1.000000, max 10.625000; Exposure Range min 13000, max 683709000;

GST_ARGUS: 1640 x 1232 FR = 29.999999 fps Duration = 33333334 ; Analog Gain range min 1.000000, max 10.625000; Exposure Range min 13000, max 683709000;

GST_ARGUS: 1280 x 720 FR = 59.999999 fps Duration = 16666667 ; Analog Gain range min 1.000000, max 10.625000; Exposure Range min 13000, max 683709000;

GST_ARGUS: 1280 x 720 FR =

[ WARN:0@217.899] global cap_gstreamer.cpp:1777 open OpenCV | GStreamer warning: Cannot query video position: status=0, value=-1, duration=-1


In [8]:
#camera.stop()

In [7]:
control_utils.resetCameraPosition()